In [1]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import json
import random

In [2]:
# Dataframe for hour, name, avg_rating, enrollment, difficulty, link, site 

def coursera_scraper (keyword):
    
    keyword = re.sub(" ","%20",keyword)
    url = f"https://www.coursera.org/search?query={keyword}&index=prod_all_products_term_optimization&entityTypeDescription=Courses&allLanguages=English"
    page = list(range(2,30))
    urls = []
    urls.append(url)
    for i in page:
        urls.append("https://www.coursera.org/search?query=" + keyword + "&page="+ str(i) + "&index=prod_all_products_term_optimization&entityTypeDescription=Courses&allLanguages=English")


    hour = []
    name = []
    avg_rating = []
    enrollment = []
    difficulty = []
    link = []
    site = []

    for url in urls:
        try:
            website_url = requests.get(url, \
                                  headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64)'}).text
            soup_html = BeautifulSoup(website_url, "lxml")
            courses = soup_html.findAll("script")
            html = courses[4]
            json_object = json.loads(html.contents[0])
            t =  json_object["props"]["pageProps"]["resultsState"]
            for i in t:
                x = i['content']["_rawResults"][0]["hits"]
                for j in x:
                    if("avgLearningHours" in j):
                        hour.append(j["avgLearningHours"])
                        name.append(j["name"])
                        avg_rating.append(j["avgProductRating"])
                        enrollment.append(j["enrollments"])
                        difficulty.append(j["productDifficultyLevel"])
                        link.append('https://www.coursera.org' + j["objectUrl"])
                        site.append('cousera')
            
            time.sleep(random.randint(5,15))
            print(url)
        except Exception as e:
            print (e)

    course_df = pd.DataFrame({'hour': hour, 'name': name, 'avg_rating': avg_rating, 'enrollment': enrollment, 
                              'difficulty': difficulty, 'link': link, 'site': site})

    return course_df

In [8]:
## Dataframe for duration and review
def duration_reviews (dataframe):  
    duration = []
    reviews = []
    
    for url in dataframe['link']:
        try:

            website_url = requests.get(url, \
                                      headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64)'}).text
            soup_html = BeautifulSoup(website_url, "lxml")
            
            # duration
            text = soup_html.find_all('div', {"class":"_16ni8zai m-b-0"})
            time_frame = text[4].getText()
            length = re.findall('[\d]{1,2} \S+', time_frame)
            duration.append(length)
            
            # review
            revs = soup_html.find_all('p', {"class":"rc-TopReviewsListItem__comment"})
            review = ''
            for i in revs:
                review += str(i.getText())
            reviews.append(review)
            
            # sleep
            time.sleep(random.randint(5,10))
            print(url)

        except Exception as e:
            duration.append('None indicated')
            reviews.append('None indicated')
            print (e)
    
    return pd.DataFrame({'duration': duration, 'review': reviews})

In [4]:
def make_csv(keyword):
    # Input: keyword is string 
    df = coursera_scraper(keyword)
    dur_rev =  duration_reviews(df)
    combined_df = pd.concat([df, dur_rev], axis = 1)
    combined_df.to_csv('coursera_' + str(keyword) + '.csv')
    return combined_df

In [11]:
# SAMPLE SCRAPING CODE

make_csv('Mathematics')

https://www.coursera.org/search?query=Mathematics&index=prod_all_products_term_optimization&entityTypeDescription=Courses&allLanguages=English
https://www.coursera.org/search?query=Mathematics&page=2&index=prod_all_products_term_optimization&entityTypeDescription=Courses&allLanguages=English
https://www.coursera.org/search?query=Mathematics&page=3&index=prod_all_products_term_optimization&entityTypeDescription=Courses&allLanguages=English
https://www.coursera.org/search?query=Mathematics&page=4&index=prod_all_products_term_optimization&entityTypeDescription=Courses&allLanguages=English
https://www.coursera.org/search?query=Mathematics&page=5&index=prod_all_products_term_optimization&entityTypeDescription=Courses&allLanguages=English
https://www.coursera.org/search?query=Mathematics&page=6&index=prod_all_products_term_optimization&entityTypeDescription=Courses&allLanguages=English
https://www.coursera.org/search?query=Mathematics&page=7&index=prod_all_products_term_optimization&entityTy

https://www.coursera.org/learn/introduction-to-calculus
https://www.coursera.org/specializations/mathematics-for-data-science
https://www.coursera.org/learn/logic-introduction
https://www.coursera.org/specializations/discrete-mathematics
https://www.coursera.org/learn/game-theory-1
https://www.coursera.org/specializations/advanced-statistics-data-science
https://www.coursera.org/learn/complex-analysis
https://www.coursera.org/learn/machine-learning-data-lifecycle-in-production
https://www.coursera.org/specializations/algorithms-for-battery-management-systems
https://www.coursera.org/specializations/financialtechnology
https://www.coursera.org/specializations/excel-data-analytics-visualization
https://www.coursera.org/specializations/michiganux
https://www.coursera.org/specializations/statistics
https://www.coursera.org/specializations/electronic-music-production
https://www.coursera.org/specializations/anatomy
https://www.coursera.org/learn/classification-vector-spaces-in-nlp
https://w

,hour,name,avg_rating,enrollment,difficulty,link,site,duration,review
0,23.0,Introduction to Mathematical Thinking,4.797148,350390,Intermediate,https://www.coursera.org/learn/mathematical-th...,cousera,[],An awesome course. Very easy to follow at the ...
1,NaN,Algebra: Elementary to Advanced,4.800000,1275,Beginner,https://www.coursera.org/specializations/algeb...,cousera,[4 months],
2,NaN,Mathematics for Machine Learning,4.581962,271487,Beginner,https://www.coursera.org/specializations/mathe...,cousera,[4 months],
3,35.0,Introduction to Calculus,4.817722,121876,Intermediate,https://www.coursera.org/learn/introduction-to...,cousera,[],I have completed quite a few courses on Course...
4,NaN,Mathematics for Data Science,4.360561,58578,Beginner,https://www.coursera.org/specializations/mathe...,cousera,[6 months],
...,...,...,...,...,...,...,...,...,...
135,24.0,Introduction to Logic,4.448393,174278,Intermediate,https://www.coursera.org/learn/logic-introduction,cousera,[],Somehow difficult in the last several weeks. A...
136,NaN,Introduction to Discrete Mathematics for Compu...,4.459556,144612,Beginner,https://www.coursera.org/specializations/discr...,cousera,[6 months],
137,20.0,Game Theory,4.620737,338096,Beginner,https://www.coursera.org/learn/game-theory-1,cousera,[],Great ! Interesting and abound at the same tim...
138,NaN,Advanced Statistics for Data Science,4.449740,89303,Advanced,https://www.coursera.org/specializations/advan...,cousera,[5 months],
